In [1]:
import os.path

import parameter
import query
import time
import copy
from model import Model
from station import Station
import numpy as np
import pandas as pd
from datetime import datetime , timedelta
import matplotlib.pyplot as plt

In [2]:
# parameter.get_station_information(True)

In [26]:
stations = pd.read_csv('data/stations.csv', index_col=0)
tph = 4
path = f'data/station_data/test_whitelist/v1/tph_{tph}/'
if not os.path.exists(path):
    os.makedirs(path)
    
station_information = pd.read_csv('data/station_data/status_at_time/2024_6_18_10:4.csv', index_col=0)
whitelist = []
for station in station_information['name']:
    if station_information.loc[station_information['name'] == station, 'operating'].values[0]:
        whitelist.append(station)
    
# whitelist = ['Allen St & Rivington St', 'Allen St & Stanton St', '1 Ave & E 16 St', 'Mercer St & Spring St', 'Mercer St & Bleecker St', '9 Ave & W 18 St', 'E 103 St & Lexington Ave', '1 Ave & E 6 St', 'Norfolk St & Broome St', 'E 1 St & 1 Ave', 'S 4 St & Wythe Ave', 'Lafayette St & Jersey St', 'Broadway & Berry St', 'S 5 Pl & S 5 St', 'E 5 St & Cooper Sq', 'Howard St & Lafayette St', 'E 32 St & Park Ave', 'Greenwich St & Hubert St', 'W 27 St & 7 Ave', 'Vesey Pl & River Terrace']

now = time.time()
station_list = {}
start_date = query.get_datetime(2023, 5, 1, 0, 0, 0)
end_date = query.get_datetime(2023, 6, 30, 0, 0, 0)
# for station in whitelist:
#     station_list[station] = parameter.get_station(station, station_information = station_information,
#                                                   start_date=start_date, end_date=end_date, whitelist=whitelist, tph=4)
#     print(station, 'done')
#     parameter.pickle_station(station_list[station], path)

0
0
0
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [4]:
station_list1 = {}
path1 = f'data/station_data/test_whitelist/v1/'
tph = 12
for station in whitelist:
    station_list1[station] = parameter.get_pickle_station(station, path1)
    station_list1[station].refine_by_3()
station_list0 = copy.deepcopy(station_list1)
station_list2 = copy.deepcopy(station_list1)
model_0 = Model(station_names=whitelist, stations_dict=station_list0, in_transit=[], tph=tph)
model_0.init_state(path='data/station_data/status_at_time/2024_6_17_14:6.csv', time=timedelta(hours=14))
model_1 = Model(station_names=whitelist, stations_dict=station_list1, in_transit=[], tph=tph)
model_1.init_state(path='data/station_data/status_at_time/2024_6_17_14:6.csv', time=timedelta(hours=14))
model_2 = Model(station_names=whitelist, stations_dict=station_list2, in_transit=[], tph=tph)
model_2.init_state(path='data/station_data/status_at_time/2024_6_17_15:5.csv', time=timedelta(hours=15))


In [5]:
model_1.truncate_transitions()

In [17]:
model_1.sim()
station_bikes = 0
for station in model_1.stations_dict.values():
    print(station.name, 'has', station.curr_bikes, 'bikes')
    station_bikes += station.curr_bikes
print(len(model_1.in_transit), 'bikes in transit')
print(station_bikes, 'station bikes')
print(len(model_1.in_transit)+station_bikes, 'total bikes')
print('time:', model_1.curr_time, 'total_trips:', model_1.total_trips, 'failures:', model_1.failures)
print(model_1.mean_sq_error(model_2.stations_dict))
print(model_0.mean_sq_error(model_2.stations_dict))

Allen St & Rivington St has 13 bikes
Allen St & Stanton St has 9 bikes
1 Ave & E 16 St has 2 bikes
Mercer St & Spring St has 62 bikes
Mercer St & Bleecker St has 61 bikes
9 Ave & W 18 St has 73 bikes
E 103 St & Lexington Ave has 18 bikes
1 Ave & E 6 St has 6 bikes
Norfolk St & Broome St has 40 bikes
E 1 St & 1 Ave has 38 bikes
S 4 St & Wythe Ave has 2 bikes
Lafayette St & Jersey St has 51 bikes
Broadway & Berry St has 3 bikes
S 5 Pl & S 5 St has 27 bikes
E 5 St & Cooper Sq has 33 bikes
Howard St & Lafayette St has 34 bikes
E 32 St & Park Ave has 4 bikes
Greenwich St & Hubert St has 3 bikes
W 27 St & 7 Ave has 0 bikes
Vesey Pl & River Terrace has 39 bikes
42 bikes in transit
518 station bikes
560 total bikes
time: 15:00:00 total_trips: 181 failures: 11
26.75
9.95


In [18]:
model_1.sim_trips()
for station in whitelist:
    print('init', model_0.stations_dict[station].curr_bikes, 'pred', model_1.stations_dict[station].curr_bikes, 'res', model_2.stations_dict[station].curr_bikes, model_0.stations_dict[station].rate[model_1.curr_tick])
print(model_1.mean_sq_error(model_2.stations_dict))
print(model_0.mean_sq_error(model_2.stations_dict))


init 14 pred 15 res 11 0.9185185185185185
init 9 pred 13 res 12 1.0222222222222221
init 1 pred 2 res 0 1.0
init 61 pred 62 res 53 0.8444444444444443
init 65 pred 62 res 59 0.9777777777777776
init 75 pred 73 res 76 1.1555555555555554
init 20 pred 20 res 21 0.2962962962962963
init 14 pred 8 res 20 0.9037037037037037
init 32 pred 40 res 33 1.3629629629629627
init 34 pred 38 res 33 0.9703703703703703
init 5 pred 2 res 7 0.5555555555555556
init 57 pred 51 res 55 1.311111111111111
init 5 pred 3 res 6 0.5259259259259259
init 35 pred 27 res 34 0.5185185185185185
init 35 pred 33 res 33 0.5555555555555556
init 34 pred 36 res 33 0.6666666666666666
init 8 pred 5 res 4 1.488888888888889
init 5 pred 5 res 5 1.7703703703703701
init 4 pred 1 res 4 1.325925925925926
init 47 pred 40 res 44 1.1703703703703703
23.65
9.95


In [ ]:
# now = time.time()
# stations = pd.read_csv('data/stations.csv', index_col=0)
# station_list = {}
# for station in stations['station_name']:
#     station_list[station] = parameter.get_pickle_station(station)
# remove = {}
# for station in station_list.values():
#     for i in range(len(station.transition)):
#         if sum(station.transition[i]) == 0:
#             station.transition[i][0] = 1
# print(time.time() - now)